In [ ]:
!git clone 

In [ ]:
!python3 train.py

In [ ]:
!python3 sample.py

In [ ]:
!python3 simple_diffusion_model.py

In [ ]:
import argparse
from train import train

parser = argparse.ArgumentParser(description='Arguments for ddpm training')
parser.add_argument('--config', dest='config_path',
                    default='config.yaml', type=str)
args = parser.parse_args()
train(args)

In [ ]:
from sample import infer
infer(args)

In [ ]:
from simple_diffusion_model import *
from utils import *

if __name__ == '__main__':

    device = "cuda" if torch.cuda.is_available() else "cpu"

    # Download Dataset
    # Transform to tensor and normalize
    transform = transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    # Load CIFAR-10 dataset
    data = torchvision.datasets.CIFAR10(root=".", train=True, download=True, transform=transform)

    # Display the images
    #show_images(data)

    # Image Size
    IMG_SIZE = 32
    BATCH_SIZE = 128

    # Define beta schedule
    T = 300
    betas = linear_beta_schedule(timesteps=T)

    # Pre-calculate different terms for closed form
    alphas = 1. - betas
    alphas_cumprod = torch.cumprod(alphas, axis=0)
    alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)
    sqrt_recip_alphas = torch.sqrt(1.0 / alphas)
    sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
    sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - alphas_cumprod)
    posterior_variance = betas * (1. - alphas_cumprod_prev) / (1. - alphas_cumprod)


    # load data
    data = load_transformed_dataset()
    dataloader = DataLoader(data, batch_size=BATCH_SIZE, shuffle=True, drop_last=True)

    # Forward Diffusion test
    # Simulate forward diffusion
    """image = next(iter(dataloader))[0]

    plt.figure(figsize=(15,15))
    plt.axis('off')
    num_images = 10
    stepsize = int(T/num_images)

    for idx in range(0, T, stepsize):
        t = torch.Tensor([idx]).type(torch.int64)
        plt.subplot(1, num_images+1, int(idx/stepsize) + 1)
        img, noise = forward_diffusion_sample(image, t)
        show_tensor_image(img)"""
    
    model = SimpleUnet()
    print("Num params: ", sum(p.numel() for p in model.parameters()))
    #model

    def get_loss(model, x_0, t):
        x_noisy, noise = forward_diffusion_sample(x_0, t, device)
        noise_pred = model(x_noisy, t)
        return F.l1_loss(noise, noise_pred)
    
    model.to(device)
    optimizer = Adam(model.parameters(), lr=0.001)
    epochs = 100

    for epoch in range(epochs):
        for step, batch in enumerate(dataloader):
          optimizer.zero_grad()

          t = torch.randint(0, T, (BATCH_SIZE,), device=device).long()
          loss = get_loss(model, batch[0], t)
          loss.backward()
          optimizer.step()

          if epoch % 5 == 0 and step == 0:
            print(f"Epoch {epoch} | step {step:03d} Loss: {loss.item()} ")
            #sample_plot_image()
    torch.save(model.state_dict())